In [32]:
!pip install vanna[chromadb,openai,mysql]

In [34]:
from vanna.openai.openai_chat import OpenAI_Chat
from vanna.chromadb.chromadb_vector import ChromaDB_VectorStore
import openai
import os

API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = API_KEY

In [36]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)
        
vn = MyVanna(config={'model': 'gpt-3.5-turbo'})

20240626 203146 INFO Collection documentation is not created.
20240626 203146 INFO Collection ddl is not created.
20240626 203146 INFO Collection sql is not created.


In [37]:
pip install vanna[postgres]

Note: you may need to restart the kernel to use updated packages.


In [38]:
vn.connect_to_postgres(host='localhost', dbname='classicmodels', user='nl2sql', password='Lehnex12345', port='5432')

In [39]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
vn.train(plan=plan)

20240626 203219 WARNING Add of existing embedding ID: 54f2c91c-67a0-548c-a269-1e8a1e99e23e-doc
20240626 203219 WARNING Insert of existing embedding ID: 54f2c91c-67a0-548c-a269-1e8a1e99e23e-doc
20240626 203219 WARNING Add of existing embedding ID: be50a997-35da-534d-ac8a-3e03cdb4acab-doc
20240626 203219 WARNING Insert of existing embedding ID: be50a997-35da-534d-ac8a-3e03cdb4acab-doc
20240626 203219 WARNING Add of existing embedding ID: 852c5655-6246-5158-adbc-de9d1d512326-doc
20240626 203219 WARNING Insert of existing embedding ID: 852c5655-6246-5158-adbc-de9d1d512326-doc
20240626 203219 WARNING Add of existing embedding ID: 44fbe607-fd06-580f-a97e-c9593853139c-doc
20240626 203219 WARNING Insert of existing embedding ID: 44fbe607-fd06-580f-a97e-c9593853139c-doc
20240626 203219 WARNING Add of existing embedding ID: a6de240c-48f0-5b2f-9c1a-2b1b12aca8b5-doc
20240626 203219 WARNING Insert of existing embedding ID: a6de240c-48f0-5b2f-9c1a-2b1b12aca8b5-doc
20240626 203219 WARNING Add of exis

In [20]:
# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )
""")

# Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="Our business defines OTIF score as the percentage of orders that are delivered on time and in full")

# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="SELECT * FROM nl2sql.employee WHERE firstname = 'Melissa' and lastname='Brown'")

20240625 235503 DEBUG Starting component PersistentLocalHnswSegment
20240625 235503 DEBUG Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'The user will give you SQL and you will try to guess what the business question this query is answering. Return just the question without any additional explanation. Do not reference the table name in the question.'}, {'role': 'user', 'content': "SELECT * FROM nl2sql.employee WHERE firstname = 'Melissa' and lastname='Brown'"}], 'model': 'gpt-3.5-turbo', 'max_tokens': 500, 'stop': None, 'temperature': 0.7}}
20240625 235503 DEBUG Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
20240625 235503 DEBUG connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None


Adding ddl: 
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )

Adding documentation....
Using model gpt-3.5-turbo for 73.0 tokens (approx)


20240625 235503 DEBUG connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001859DB71490>
20240625 235503 DEBUG start_tls.started ssl_context=<ssl.SSLContext object at 0x000001859CF822A0> server_hostname='api.openai.com' timeout=5.0
20240625 235503 DEBUG start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001859DA60810>
20240625 235503 DEBUG send_request_headers.started request=<Request [b'POST']>
20240625 235503 DEBUG send_request_headers.complete
20240625 235503 DEBUG send_request_body.started request=<Request [b'POST']>
20240625 235503 DEBUG send_request_body.complete
20240625 235503 DEBUG receive_response_headers.started request=<Request [b'POST']>
20240625 235504 DEBUG receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 26 Jun 2024 04:55:02 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'

Question generated with sql: Which employee has the first name 'Melissa' and the last name 'Brown'? 
Adding SQL...


'48a8630a-ff1e-5baa-be7a-89d9995d5176-sql'

In [52]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type
0,001a50eb-50e3-5ce2-9b95-65e38d15c8a3-doc,None,The following columns are in the pg_stat_bgwri...,documentation
1,001f9aa2-7bae-51a9-9387-f1305fddb470-doc,None,The following columns are in the pg_stat_progr...,documentation
2,08069f10-4325-553d-8ea5-3919bf2ccea8-doc,None,The following columns are in the lineitem tabl...,documentation
3,0843438e-7e8f-5c6e-836e-7b24a3afe7a0-doc,None,The following columns are in the pg_seclabel t...,documentation
4,09909ede-ce14-5099-b5b9-2d6c57224270-doc,None,The following columns are in the supplier tabl...,documentation
...,...,...,...,...
216,fa1d1403-ef64-50e3-b938-f7609cd74f3b-doc,None,The following columns are in the pg_inherits t...,documentation
217,fab044f2-ed5f-527f-82b1-7790cff22633-doc,None,The following columns are in the column_udt_us...,documentation
218,fbffad7e-a5b6-5e51-9a82-87d3d806330e-doc,None,The following columns are in the pg_ts_parser ...,documentation
219,fdcc5489-446f-505b-8327-6d32e4b2f007-doc,None,The following columns are in the pg_conversion...,documentation


In [51]:
# You can remove training data if there's obsolete/incorrect information. 
vn.remove_training_data(id='4aafa9de-6443-5409-86f9-5d581327d6a0-ddl')

True

In [53]:
vn.train(documentation="Sales data is in nl2sql.salesorder table")

20240626 203528 WARNING Add of existing embedding ID: 25e2a8db-0aee-5a7f-97c1-54490c944e00-doc
20240626 203528 WARNING Insert of existing embedding ID: 25e2a8db-0aee-5a7f-97c1-54490c944e00-doc


Adding documentation....


'25e2a8db-0aee-5a7f-97c1-54490c944e00-doc'

In [54]:
vn.train(documentation="Sales details are in nl2sql.lineitem table")

Adding documentation....


'8c113eb2-e975-5291-ba9c-7040ceee63ee-doc'

In [57]:
vn.ask(question="what are the top 5 customer by sales")

20240626 203728 WARNING Number of requested results 10 is greater than number of elements in index 0, updating n_results = 0
20240626 203728 WARNING Number of requested results 10 is greater than number of elements in index 0, updating n_results = 0


SQL Prompt: [{'role': 'system', 'content': "You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Additional Context \n\nSales details are in nl2sql.lineitem table\n\nSales details is in nl2sql.lineitem table\n\nSales data is in nl2sql.salesorder table\n\nJoin the nl2sql.Salesorder table and nl2sql.lineitem table to get sales details\n\nThe following columns are in the customer table in the classicmodels database:\n\n|      | table_catalog   | table_schema   | table_name   | column_name     | data_type         |\n|-----:|:----------------|:---------------|:-------------|:----------------|:------------------|\n|  369 | classicmodels   | public         | customer     | billingaddress  | text              |\n|  683 | classicmodels   | public         | customer     | phone           | character varying |\n|  968 | classicmodels   | publ

(None, None, None)

In [29]:
vn.train(documentation="Join the nl2sql.Salesorder table and nl2sql.lineitem table to get sales details")

Adding documentation....


'61b299f2-1b80-501e-9081-31b0f0e263bc-doc'

In [58]:
vn.ask(question="who are the top 5 employees by salary")

20240626 203933 WARNING Number of requested results 10 is greater than number of elements in index 0, updating n_results = 0
20240626 203933 WARNING Number of requested results 10 is greater than number of elements in index 0, updating n_results = 0


SQL Prompt: [{'role': 'system', 'content': "You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Additional Context \n\nThe following columns are in the employee table in the classicmodels database:\n\n|      | table_catalog   | table_schema   | table_name   | column_name   | data_type         |\n|-----:|:----------------|:---------------|:-------------|:--------------|:------------------|\n|  122 | classicmodels   | nl2sql         | employee     | hiredate      | date              |\n|  613 | classicmodels   | nl2sql         | employee     | employeeid    | integer           |\n|  849 | classicmodels   | nl2sql         | employee     | phone         | character varying |\n| 1052 | classicmodels   | nl2sql         | employee     | position      | character varying |\n| 1297 | classicmodels   | nl2sql         | employee     | salary

(None, None, None)